# Asana API Tutorial Notebook

**What this Notebook Does:**
- Shows how to authenticate and interact with Asana via our custom API layer.
- Demonstrates how to fetch tasks and comments for a specified project and time period.
- Computes simple statistics (e.g., tasks created, tasks completed, number of comments).
- Includes unit tests to ensure the code runs correctly.

**Prerequisites:**
- A valid Asana Personal Access Token (PAT).

**Instructions:**
1. Recommended to your `ASANA_ACCESS_TOKEN` environment variable before running.




In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import datetime
import logging

import helpers.hdbg as hdbg
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import utils

In [15]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

hprint.config_notebook()

## Define Config

Here we define all parameters in a single `config` dictionary.
ou can easily modify:
- The `project_id` to analyze a different project.
- The `start_date` and `end_date` to change the timeframe.

In [16]:
today = datetime.datetime.now()
one_month_ago = today - datetime.timedelta(days=30)
config = {
    # Replace with a valid project ID from your Asana workspace.
    "project_id": ["1208279350109582"],
    "start_date": one_month_ago.isoformat(),
    "end_date": today.isoformat(),
    "access_token": "2/1208871906331279/1208966663406154:1c6f6b89083e73c22241670b11bf00ba",
}

## Intialize asana client

In [17]:
client = utils.AsanaClient(access_token=config["access_token"])

## Fetching Task Data

 Using the parameters in `config`, we’ll fetch:
 - Tasks created within the `start_date` and `end_date`
 - Tasks completed within the same range

 The `fetch_tasks()` function returns a DataFrame with columns like:
 - `task_id`
 - `name`
 - `assignee`
 - `created_at`
 - `completed_at`


In [22]:
# Fetch tasks created in the given period.
tasks_df = utils.fetch_tasks(
    client,
    project_ids=config["project_id"],
    start_date=config["start_date"],
    end_date=config["end_date"],
)
# Fetch tasks completed in the given period.
_LOG.info(
    "Created_taaks_df = \n%s", hpandas.df_to_str(tasks_df, log_level=logging.INFO)
)

,task_id,assignee,completed,completed_at,created_at,name,project_id,project_status,task_status
0,1208871906331267,None,False,NaT,2024-11-29 13:22:25.113000+00:00,Create specs for all the class projects,1208279350109582,Active,Incomplete
1,1208871906331269,None,False,NaT,2024-11-29 13:22:31.777000+00:00,Create sign up Gsheet for the projects,1208279350109582,Active,Incomplete
2,1208871906331273,None,False,NaT,2024-11-29 13:23:06.880000+00:00,Recover / reorg past years tests,1208279350109582,Active,Incomplete
,...,...,...,...,...,...,...,...,...
14,1208751109010015,Shaunak Dhande,False,NaT,2024-11-12 20:31:13.332000+00:00,Allen Mathews <mathew10@umd.edu>,1208279350109582,Active,Incomplete
15,1208778585230952,None,False,NaT,2024-11-16 19:02:56.602000+00:00,Vikas Patel <vikas01@umd.edu>,1208279350109582,Active,Incomplete
16,1208778585230961,None,False,NaT,2024-11-18 02:53:10.073000+00:00,Abhinav Maheshwari <abhinavm@umd.edu>,1208279350109582,Active,Incomplete


INFO  Created_taaks_df = 
None


## Fetching Comments (Stories)
We now fetch comments for the tasks that were created or completed in the time window.
`get_task_comments`:
- Takes a list of task_ids.
- Returns a DataFrame with `task_id`, `comment_text`, `comment_author`, `comment_created_at`.


In [28]:
task_ids = (
    tasks_df["task_id"].tolist() if not tasks_df.empty else []
)
tasks_comments_df = utils.fetch_comments(client, task_ids)
_LOG.info(
    "Comments df = \n %s",
    hpandas.df_to_str(tasks_comments_df, log_level=logging.INFO),
)

,task_id,text,author,created_at
0,1208751109009920,"Respected Professor Giacinto Paolo Saggese\nUniversity of Maryland, College Park\n\n\nI hope this email finds you well. My name is Dhruv Gupta, a Master's student in Data Science at the University of Maryland, College Park. I am reaching out to express my interest in any assistantship opportunities within your department. With a strong background in data science, software engineering, and business analytics, I am confident in my ability to make meaningful contributions as a Research, Graduate, or Teaching Assistant.\n\nMy motivation for data science is deeply rooted in the intersection of mathematics, statistics, computer science, and domain expertise, captivating my intellectual curiosity. Early exposure to data analysis courses during my academic journey and my innate inquisitiveness ignited my passion for distilling knowledge from raw information. The ability of data science to uncover hidden patterns and trends, thereby synthesizing disparate data points into coherent narratives, became a source of enduring fascination and intellectual engagement. To further explore this territory, I completed my undergraduate studies at the University of Petroleum and Energy Studies, where I earned a Bachelor of Technology - Computer Science with a specialization in Business Analytics and Optimization.\n\nWith a background in Computer Science and a specialization in Business Analytics, I have honed skills in data analysis, statistical modeling, and machine learning, particularly through my work at PwC India. There, I leveraged tools like Python's Pandas and R to analyze complex datasets, ensuring high data quality and accuracy. I also evaluated model performance using key metrics, gaining valuable experience in extracting meaningful insights and contributing to strategic decision-making.\n\nMy previous role as a Software Development Engineer at PeopleStrong enabled me to work on web applications using Flutter and Dart, enhancing my software development and problem-solving a...",GP Saggese,2024-11-12T20:25:49.263Z
1,1208751109009926,"Dear Professor,\nI hope this email finds you well. My name is Riya Ambolkar, and I am currently pursuing a Master of Science in Information Systems at the University of Maryland, expected to graduate in December 2025. I am reaching out to inquire about any available Teaching Assistant or Graduate Assistant positions within the Science Academy.\nThroughout my academic and professional career, I have gained experience in data analysis, machine learning, and programming, skills that I am eager to apply in an academic setting. As an Information Technology Analyst at Kansai Nerolac Paints, I worked on projects involving predictive modeling, data visualization, and the optimization of business processes, leveraging technologies such as Python, SQL, Tableau, and Power BI. I also have a solid foundation in academic research, having published a paper on a hotel recommendation system using modified BERT in IEEE Xplore, as well as delivering a presentation at the International Conference on Artificial Intelligence and Smart Systems.\nMy experience and technical proficiency make me confident that I can contribute effectively to any research or teaching activities within the Science Academy. I would greatly appreciate the opportunity to assist in the department and expand my academic knowledge and research experience.\nIf there are any openings or further details about the application process, I would be grateful for your guidance. I have attached my resume and the published paper for your reference and would be thrilled to discuss how I can contribute to your team.\nThank you for your time and consideration. I look forward to hearing from you soon.\nBest regards,\nRiya Ambolkar\nUID - 121093257",GP Saggese,2024-11-12T20:26:56.381Z
2,1208751109009934,"Dear Professor Saggese,\n\nI hope this email finds you well. I am currently pursuing my Master of Science in Applied Machi

INFO  Comments df = 
 None


## Computing Statistics
We'll compute:
- Number of tasks created in the period.
- Number of comments on tasks created in the period.

In [30]:
num_created_tasks = len(tasks_df)
num_comments_on_created = len(tasks_comments_df)
_LOG.info("Number of tasks created in the period: %s", num_created_tasks)
_LOG.info("Number of comments on created tasks: %s", num_comments_on_created)

INFO  Number of tasks created in the period: 17
INFO  Number of comments on created tasks: 11


## Statistics for All Users

We can aggregate by user (assignee) to see how many tasks each user created or completed.

**Tasks Created per User**:
If `created_tasks_df` includes `assignee`, we can group by that column.


In [31]:
if not tasks_df.empty and "assignee" in tasks_df.columns:
    tasks_created_by_user = (
        tasks_df.groupby("assignee")["task_id"].count().reset_index()
    )
    tasks_created_by_user.columns = ["assignee", "tasks_created_count"]
    print("Tasks Created by User:")
    print(tasks_created_by_user)
else:
    print("No tasks created or 'assignee' information not available.")

Tasks Created by User:
                  assignee  tasks_created_count
0         Krishna P Taduri                    1
1  Prahar Kaushikbhai Modi                    1
2           Shaunak Dhande                    8


**Tasks Completed per User**:

Similarly, for completed tasks:

In [33]:
completed_tasks_df = tasks_df[tasks_df["task_status"] == "Completed"]
if not completed_tasks_df.empty and "assignee" in completed_tasks_df.columns:
    tasks_completed_by_user = (
        completed_tasks_df.groupby("assignee")["task_id"].count().reset_index()
    )
    tasks_completed_by_user.columns = ["assignee", "tasks_completed_count"]
    print("Tasks Completed by User:")
    print(tasks_completed_by_user)
else:
    print("No tasks completed or 'assignee' information not available.")

No tasks completed or 'assignee' information not available.


**Comments per User**:

For comments, we have `author`. We can see how many comments each user made during this period for both created and completed tasks.


In [35]:
if not tasks_comments_df.empty and "author" in tasks_comments_df.columns:
    comments_by_user = (
        tasks_comments_df.groupby("author")["task_id"].count().reset_index()
    )
    comments_by_user.columns = ["author", "comments_count"]
    print("Comments by User:")
    print(comments_by_user)
else:
    print("No comments found or 'comment_author' information not available.")

Comments by User:
           author  comments_count
0      GP Saggese              10
1  Shaunak Dhande               1
